Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [1]:
#Update your token
STUDENT_TOKEN = 'Joaquin Casasus'

There should be no need to change the code below.

In [2]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


There is almost certainly no need to change this code, but if you must you can.

In [3]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Clobber(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):
    for row in self.get_board():
      print(row)


Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return a list or tuple of 3 elements: row, col and dir. Dir can be 'R', 'L', 'U', 'D'

In [4]:
def random_solver(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  opts = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          opts.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          opts.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          opts.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          opts.append((row, col, 'L'))
  return opts[random.randint(0, len(opts) - 1)]

This code is the one that starts the game. In different games you will need to change the class (Clobber is the one for clobber game).

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [5]:
from queue import PriorityQueue
import math
import numpy as np
from copy import copy, deepcopy

class Node():
  def __init__(self, state, game, parent = None):
    self.state = state
    self.initial_player = game.player
    self.player = game.player
    self.other_player = game.other_player()
    #self.turn = True
    self.parent = parent
    self.children = []
    self.n = 0
    #self.n_parent = self.parent.n
    self.wins = 0
    self.loses = 0
    self.untried_actions = self.legal_moves()
    self.actions_done=[]
    self.game = game
    return


  def legal_moves(self): # Returns a list with all the possible moves from a node
    board = self.state
    #turn = self.turn
    player = self.player
    other_player = self.other_player

    opts = []
    for row in range(len(board)):
      for col in range(len(board[row])):
        if board[row][col] == player:
          if row + 1 < len(board) and board[row + 1][col] == other_player:
            opts.append((row, col, 'D'))
          if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
            opts.append((row, col, 'R'))
          if row - 1 >= 0 and board[row - 1][col] == other_player:
            opts.append((row, col, 'U'))
          if col - 1 >= 0 and board[row][col - 1] == other_player:
            opts.append((row, col, 'L'))
    return opts


  def calculate_wins(self): # Calculates the difference between wins and loses in a path.
    wins = self.wins
    loses = self.loses
    return wins - loses


  def expand(self): # return child node from move (state)
    action = self.untried_actions.pop()
    self.actions_done.append(deepcopy(action))
    child_state = self.move(action)
    child_node = Node(child_state, self.game, parent=self)
    child_node.player = self.other_player
    child_node.other_player = self.player
    
    self.children.append(child_node)

    return child_node


  def move(self, action): # Takes a move from "legal_moves" and updates the state with the move.
    player = self.player
    other_player = self.other_player

    new_state = deepcopy(self.state)
    child_node = Node(new_state, self.game, parent=self)
    board = child_node.state
    row = action[0]
    col = action[1]
    dir = action[2]

    if dir == 'D':
      board[row][col] = '.'
      board[row + 1][col] = player

    elif dir == 'R':
      board[row][col] = '.'
      board[row][col + 1] = player

    elif dir == 'U':
      board[row][col] = '.'
      board[row - 1][col] = player

    elif dir == 'L':
      board[row][col] = '.'
      board[row][col - 1] = player    

    return board


  def is_terminal_node(self): # Checks if a node is terminal
    return len(self.legal_moves()) == 0


  def rollout(self): # Loop for simulation.
  # Chosing a random action, creates states with the action, looking for new actions to do over the updated states until it reaches a terminl node.
  # Then returning 1 for a win and 0 for a lost.

    current_rollout_state = deepcopy(self.state)

    currrent_rollout_player = self.player
    current_rollout_other_player = self.other_player
    opts = self.legal_moves()

    while len(opts) > 0:
      action = opts[np.random.randint(len(opts))]
      row = action[0]
      col = action[1]
      dir = action[2]

      if dir == 'D':
        current_rollout_state[row][col] = '.'
        current_rollout_state[row + 1][col] = currrent_rollout_player

      elif dir == 'R':
        current_rollout_state[row][col] = '.'
        current_rollout_state[row][col + 1] = currrent_rollout_player

      elif dir == 'U':
        current_rollout_state[row][col] = '.'
        current_rollout_state[row - 1][col] = currrent_rollout_player

      elif dir == 'L':
        current_rollout_state[row][col] = '.'
        current_rollout_state[row][col - 1] = currrent_rollout_player    

      new_currrent_rollout_player = current_rollout_other_player
      new_current_rollout_other_player = currrent_rollout_player

      currrent_rollout_player = new_currrent_rollout_player
      current_rollout_other_player = new_current_rollout_other_player

      opts = []
      for row in range(len(current_rollout_state)):
        for col in range(len(current_rollout_state[row])):
          if current_rollout_state[row][col] == currrent_rollout_player:
            if row + 1 < len(current_rollout_state) and current_rollout_state[row + 1][col] == current_rollout_other_player:
              opts.append((row, col, 'D'))
            if col + 1 < len(current_rollout_state[row]) and current_rollout_state[row][col + 1] == current_rollout_other_player:
              opts.append((row, col, 'R'))
            if row - 1 >= 0 and current_rollout_state[row - 1][col] == current_rollout_other_player:
              opts.append((row, col, 'U'))
            if col - 1 >= 0 and current_rollout_state[row][col - 1] == current_rollout_other_player:
              opts.append((row, col, 'L'))

    if currrent_rollout_player == self.initial_player:
      return 0
    
    else:
      return 1


  def backpropagate(self, result): # Goes back from the leaf node to the parent node updating the stats (wins, loses and number of simulations).
  # It goes up until there is not a parent.
    self.n += 1.
    if result == 1:
      self.wins += 1.
    else:
      self.loses += 1.

    if self.parent:
        self.parent.backpropagate(result)


  def is_fully_expanded(self): # If a node has been expanded for all its possible moves. Returns true or false.
    return len(self.untried_actions) == 0


  def best_child(self): # UCT formula to choose the best child (a node) from parent node
    c_param = 1.414213
    choices_weights = [(c.calculate_wins() / c.n) + c_param * np.sqrt((2 * np.log(self.n) / c.n)) for c in self.children]
    return self.children[np.argmax(choices_weights)]

  def best_move(self): # UCT formula to choose the best move from parent node
    c_param = 1.414213
    choices_weights = [(c.calculate_wins() / c.n) + c_param * np.sqrt((2 * np.log(self.n) / c.n)) for c in self.children]
    return [np.argmax(choices_weights)]

  def tree_policy(self): # Policy to loop through the tree.
    current_node = self

    while not current_node.is_terminal_node(): # While current node is not terminal node
      if not current_node.is_fully_expanded(): # If current node is not fully expanded, then expand the node and return one of its children
          return current_node.expand()
      else: # If it is expanded, then the current node is the child with the highest UCT.
          current_node = current_node.best_child()
    return current_node # If it is a terminal node, return current node.


  def monte_carlo(self): # Main function that decides the best child node from received action (node).
    simulation_no = 50000 #Set number of simulations 
	
    for i in range(simulation_no): # For number of simulations
        v = self.tree_policy() # This is used to expand the node
        reward = v.rollout() # Make a simulation
        v.backpropagate(reward) # Backpropagate for current simulation
    return self.actions_done[self.best_move()[0]] # Return the best move (taking into account the best child node)

In [6]:
def amazing_solver(game): # Algorithm to return move
  state = game.get_board() # Get state (board)
  pepe = Node(state, game, parent = None) # Create object for that state
  return pepe.monte_carlo() # Return best move

In [11]:
game_loop(amazing_solver, Clobber, 'clobber', multi_player=True, id=9912)

Joining game with id: 9912
Playing as X
Waiting for the other player to join...
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
Making next move...


KeyboardInterrupt: 